In [1]:
!pip install langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
! pip install langchain_community
! pip install langchain_huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.


In [5]:
from langchain_community.document_loaders import TextLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [6]:

from langchain.docstore.document import Document

# --- Step 1: Define the function to split the chat log ---

def split_chat_log_into_chunks(chat_log: str, chunk_size: int = 1, overlap: int = 0) -> list[Document]:
    """
    Splits a chat log from a string into contextual chunks.
    """
    # Split the log into individual lines based on the newline character
    turns = [line.strip() for line in chat_log.strip().split('\n') if line.strip()]
    
    documents = []
    # Loop through the turns and create overlapping chunks
    for i in range(0, len(turns), chunk_size - overlap):
        chunk_turns = turns[i : i + chunk_size]
        if not chunk_turns:
            continue
        
        chunk_text = "\n".join(chunk_turns)
        
        # Each chunk becomes a LangChain Document
        documents.append(Document(page_content=chunk_text))
        
    return documents

# --- Step 2: Load the content from your .txt file ---
file_path = '/kaggle/input/medical-dataset/Doctor_patient.txt'  
chat_content = ""

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        chat_content = f.read()
    print(f" Successfully loaded content from {file_path}")
except FileNotFoundError:
    print(f" Error: The file '{file_path}' was not found. Please check the file name and path.")
   

if chat_content:
    document_chunks = split_chat_log_into_chunks(chat_content)
    
    print(f"\nProcessed the chat log into {len(document_chunks)} documents.")

    if document_chunks:
        print("\n--- Example of the first document chunk ---")
        print(document_chunks[0].page_content)

 Successfully loaded content from /kaggle/input/medical-dataset/Doctor_patient.txt

Processed the chat log into 243485 documents.

--- Example of the first document chunk ---
Q. What should I do to reduce my weight gained due to genetic hypothyroidism?Hi. You have really done well with the hypothyroidism problem. Your levels are normal with less medications which are very good. As it is genetically induced, it is very difficult to lose weight. My advice to you is, you should focus on maintaining normal levels of TSH (thyroid-stimulating hormone) and try to remain active, having a positive outlook in life. Or else, it will become very difficult to balance your life with the symptoms of hypothyroidism. Even though your weight has not reduced, be very careful in not putting on weight here afterward. Everyday brisk walking for 1 hour. If you have body pain, alternate with exercises and walking. Avoid all kinds of junk foods, processed, bakery products, rich sweets, fatty foods, sodas, alco

In [7]:
pdf_loader = PyPDFLoader("/kaggle/input/medical-dataset/medical_book.pdf")
loaded_pdf_docs = pdf_loader.load()
    
# Split the PDF content into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
pdf_docs = text_splitter.split_documents(loaded_pdf_docs)
print(f" Successfully loaded and split PDF into {len(pdf_docs)} documents.")

 Successfully loaded and split PDF into 4110 documents.


In [8]:
all_docs = document_chunks + pdf_docs


In [9]:
len(all_docs)

247595

In [10]:
# model_name = "BAAI/bge-large-en-v1.5"
model_name='all-MiniLM-L6-v2'

model_kwargs = {'device': 'cuda'}  
encode_kwargs = {'normalize_embeddings': True} 

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

print("HuggingFaceEmbeddings initialized successfully!")


2025-08-12 10:13:58.678195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754993638.862569      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754993638.922291      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings initialized successfully!


In [11]:
db=FAISS.from_documents(all_docs,embeddings)
db

In [12]:
db.save_local("my_faiss_index")